Testing of the publicly accessible API for the production ORCID system

API 2.0 reference (in GitHub): https://github.com/ORCID/ORCID-Source/tree/master/orcid-model/src/main/resources/record_2.0

In [ ]:
import requests
import json
import datetime
import pandas

In [ ]:
# Define variables for use throughout
publicTokenResponse= {"access_token":"4a587e47-b564-412b-81a5-175ed8a92fed","token_type":"bearer","refresh_token":"236ddbf7-867f-42cc-9e08-af32526c1ba3","expires_in":631138518,"scope":"/read-public","orcid":"null"}
publicToken = publicTokenResponse["access_token"]

host = "pub.orcid.org" #member API in sandbox
headers = {"Accept":"application/json", "Authorization":"Bearer %s"%(publicToken)}


# define some extraction functions for ORCID results
def getEmployment(orcidID):
    reqString = "https://%s/v2.0/%s/employments"%(host,orcidID)
    reqGet = requests.get(reqString, headers=headers )
    return(reqGet.json())

def getPerson(orcidID):
    reqString = "https://%s/v2.0/%s/person"%(host,orcidID)
    reqGet = requests.get(reqString, headers=headers)
    return(reqGet.json())

def getPersonalDetails(orcidID):
    reqString = "https://%s/v2.0/%s/personal-details"%(host,orcidID)
    reqGet = requests.get(reqString, headers=headers)
    return(reqGet.json())

def fullRecord(orcidID):
    reqString = "https://%s/v2.0/%s/record"%(host,orcidID)
    reqGet = requests.get(reqString, headers=headers)
    return(reqGet.json())



In [34]:
# public production API testing with Karl's token/credentials

#query = "affiliation-org-name:'University+of+New+Mexico'"
#query = "ringgold-org-id:1104" #University of New Mexico
#query = "ringgold-org-id:12289" #University of New Mexico Health Sciences Center
#query = "email:*unm.edu"
query = "external-id-reference-name:'ResearcherID'" #???
headers = {"Accept":"application/json", "Authorization":"Bearer %s"%(publicToken)}

records = 200

resultList = []

for i in range(2):
    start = i * records
    baseURL = "https://%s/v2.0/search?q=%s&rows=%s&start=%s"%(host,query,records,start)

    reqGet = requests.get(baseURL, headers=headers )
    reqGet.json()['result']

    print(json.dumps(reqGet.json()))

    j = 0
    for resultItem in reqGet.json()['result']:
        returnDict = {}
        returnDict['orcidID'] = resultItem['orcid-identifier']['path']

        # get the person's name
        result = getPersonalDetails(returnDict['orcidID'])

        try:
            returnDict['familyName'] = result['name']['family-name']['value']
        except:
            returnDict['familyName'] = "ERROR"

        try:
            returnDict['givenNames'] = result['name']['given-names']['value']
        except:
            returnDict['givenNames'] = "ERROR"

        try:
            returnDict['creditName'] = result['name']['credit-name']['value']
        except:
            returnDict['creditName'] = "ERROR"





        # get the person's employment information
        result = getEmployment(returnDict['orcidID'])

        employmentList = []
        for employmentItem in result['employment-summary']:
            employmentRecord = {}
            # print(json.dumps(employmentItem, indent=4, sort_keys=True))

            try:
                employmentRecord['department_name'] = employmentItem['department-name']
            except:
                employmentRecord['department_name'] = "ERROR"

            try:
                employmentRecord['start_date'] = {'year':employmentItem['start-date']['year']['value'], 'month':employmentItem['start-date']['month']['value'],'day':employmentItem['start-date']['day']['value']}
            except:
                employmentRecord['start_date'] = {}

            try:
                employmentRecord['end_date'] = {'year':employmentItem['end-date']['year']['value'], 'month':employmentItem['end-date']['month']['value'],'day':employmentItem['end-date']['day']['value']}
            except:
                employmentRecord['end_date'] = {}

            try:
                employmentRecord['role_title'] = employmentItem['role-title']
            except:
                employmentRecord['role_title'] = "ERROR"

            try:
                employmentRecord['organization'] = employmentItem['organization']['name']
            except:
                employmentRecord['organization'] = "ERROR"

            try:
                employmentRecord['address_city'] = employmentItem['organization']['address']['city']
            except:
                employmentRecord['address_city'] = "ERROR"

            try:
                employmentRecord['address_region'] = employmentItem['organization']['address']['region']
            except:
                employmentRecord['address_region'] = "ERROR"

            try:
                employmentRecord['address_country'] = employmentItem['organization']['address']['country']
            except:
                employmentRecord['address_country'] = "ERROR"

            employmentList.append(employmentRecord)


        returnDict['employment'] = employmentList



        resultList.append(returnDict)

        #j += 1
        #if j > 10: break

#for item in resultList:
#    print(json.dumps(item, indent=4, sort_keys=True))


KeyError: 'result'

In [18]:
# flatten the resultList into a dict

flatList = []

for person in resultList:
    returnDict = {}
    #for empolyment in person['employment']:
    returnDict['familyName'] = person['familyName']
    returnDict['givenNames'] = person['givenNames']
    returnDict['creditName'] = person['creditName']
    returnDict['orcidID'] = person['orcidID']
    
    for employment in person['employment']:
        returnDict['organization'] = employment['organization']
        returnDict['department_name'] = employment['department_name']


    
        flatList.append(returnDict)




goodLuck = pandas.DataFrame(flatList)

In [75]:
print(goodLuck)

             creditName                                    department_name  \
0                 ERROR                         Mathematics and Statistics   
1                 ERROR                                 Clinical Education   
2                 ERROR                                        Archaeology   
3                 ERROR                                        Archaeology   
4                 ERROR                                      Cancer Center   
5                 ERROR                                      Cancer Center   
6                 ERROR                                      Cancer Center   
7                 ERROR                                      Cancer Center   
8                 ERROR                        Family and Consumer Science   
9                 ERROR                                            Biology   
10                ERROR                                 College of Nursing   
11                ERROR                                 College 

In [86]:
fullRecord('0000-0002-2483-5614')

{'activities-summary': {'educations': {'education-summary': [{'created-date': {'value': 1386964384543},
     'department-name': 'Anthropology (Biological)',
     'end-date': {'day': None, 'month': None, 'year': {'value': '2010'}},
     'last-modified-date': {'value': 1406005714835},
     'organization': {'address': {'city': 'Albuquerque',
       'country': 'US',
       'region': 'NM'},
      'disambiguated-organization': {'disambiguated-organization-identifier': '1104',
       'disambiguation-source': 'RINGGOLD'},
      'name': 'University of New Mexico'},
     'path': '/0000-0002-2483-5614/education/28641',
     'put-code': 28641,
     'role-title': 'PhD',
     'source': {'source-client-id': None,
      'source-name': {'value': 'Timothy R Petersen'},
      'source-orcid': {'host': 'orcid.org',
       'path': '0000-0002-2483-5614',
       'uri': 'http://orcid.org/0000-0002-2483-5614'}},
     'start-date': None,
     'visibility': 'PUBLIC'},
    {'created-date': {'value': 1386964684684}